# Phase Diagrams

<div class="alert alert-block alert-info"> In this notebook, you will scan the amplitude of the power spectrum and the bispectrum amplitudes in the parameter space. </div>

In [ ]:
# Import the standard Python packages
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Display a bar for loops

# For Latex-style labels
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r"\usepackage{bm}")

# Import CosmoFlow modules (they must be located in the same parent folder as this notebook)
from Parameters import parameters
from Theory import theory
from Solver import solver

In [ ]:
# Define the numerical i\epsilon prescription
def adiabatic(N_load, DeltaN):
    return (np.tanh((N_load + DeltaN - 1)/0.1) + 1)/2

<div class="alert alert-block alert-info"> <strong>Task:</strong> Write a function phase_diagram(m, rho) that takes the mass and the quadratic mixing strength as inputs, and outputs the final amplitudes of the two- and three-point correlator (in equilateral kinematic configuration). We will fix cs=1 for simplicity. </div>

In [ ]:
def phase_diagram(m, rho):
    
    ...
    
    return two_pt, three_pt

<div class="alert alert-block alert-info"> <strong>Task:</strong> Try to understand the following script and execute it. Caution, it might take a few minutes to run. </div>

In [ ]:
from joblib import Parallel, delayed # Needed for parallelisation
import time # Record the running time

n_pt = 10
m = np.logspace(-1, 1, n_pt)
rho = np.logspace(-1, 1, n_pt)

def process(i):
    two_pt = []
    three_pt = []
    for j in range(n_pt):
        C2, C3 = phase_diagram(m = m[i], rho = rho[j])
        two_pt.append(C2)
        three_pt.append(C3)
    return two_pt, three_pt

start_time = time.time() # Start recording runtime
X = Parallel(n_jobs = 6)(delayed(process)(i) for i in range(n_pt)) # n_jobs is the number of cores
print("--- %s seconds ---" % (time.time() - start_time)) # Print runtime

# Define 2D arrays for correlators
two_pt = np.ones((n_pt, n_pt))
three_pt = np.ones((n_pt, n_pt))

# Reconstructing correlators from the parallelisation output
for i in range(n_pt):
    for j in range(n_pt):
        two_pt[i, j] = X[i][0][j]
        three_pt[i, j] = X[i][1][j]
        
# Save files
np.save("two_pt.npy", two_pt)
np.save("three_pt.npy", three_pt)

<div class="alert alert-block alert-info"> <strong>Task:</strong> Execute the following script to show the results for the power spectrum. Comment. </div>

In [ ]:
import matplotlib
from matplotlib import cm
from pylab import contour, clabel

# Define figure
fig = plt.figure()
ax = fig.add_subplot()

ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlim((np.min(m), np.max(m)))
ax.set_ylim((np.min(rho), np.max(rho)))

X, Y = np.meshgrid(m, rho)
TwoPt = np.load("two_pt.npy")

im = ax.pcolormesh(X, Y, np.transpose(TwoPt), cmap = "coolwarm")
cb = plt.colorbar(im, shrink = 0.8)
cb.set_ticks([1, 3, 5])
ax.set_xlabel("$m/H$", fontsize = 12)
ax.set_ylabel("$\\rho/H$", fontsize = 12)
ax.set_title(r"$\langle\varphi_{\bm{k}_1} \varphi_{\bm{k}_2}\rangle'$", y = 1.05)


<div class="alert alert-block alert-info"> <strong>Task:</strong> Execute the following script to show the results for the equilateral bispectrum. Comment. </div>

In [ ]:
import matplotlib
from matplotlib import cm
from pylab import contour, clabel

# Define figure
fig = plt.figure()
ax = fig.add_subplot()

ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlim((np.min(m), np.max(m)))
ax.set_ylim((np.min(rho), np.max(rho)))

X, Y = np.meshgrid(m, rho)
ThreePt = np.load("three_pt.npy")

im = ax.pcolormesh(X, Y, np.transpose(ThreePt)/10, cmap = "coolwarm")
cb = plt.colorbar(im, shrink = 0.8)
cb.set_label("$\\times 10$", fontsize = 15, rotation = 0, y = 1.12, labelpad = -15)
cb.set_ticks([0, 1, 2])
ax.set_xlabel("$m/H$", fontsize = 12)
ax.set_ylabel("$\\rho/H$", fontsize = 12)
ax.set_title(r"$\langle\varphi_{\bm{k}_1} \varphi_{\bm{k}_2} \varphi_{\bm{k}_3}\rangle'$", y = 1.05)

